In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from scipy import *
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy
from scipy import spatial
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
STOP_WORDS = stopwords.words("english")
from sklearn.model_selection import train_test_split
import xgboost as xgb
import sklearn.metrics as classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import precision_recall_curve, auc, roc_curve ,f1_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df= pd.read_csv("/content/drive/MyDrive/FYP/train.csv") # importing the dataset

df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
df.head(20)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [ ]:
df.describe()

,id,qid1,qid2,is_duplicate
count,404290.000000,404290.000000,404290.000000,404290.000000
mean,202144.500000,217243.942418,220955.655337,0.369198
std,116708.614503,157751.700002,159903.182629,0.482588
min,0.000000,1.000000,2.000000,0.000000
25%,101072.250000,74437.500000,74727.000000,0.000000
50%,202144.500000,192182.000000,197052.000000,0.000000
75%,303216.750000,346573.500000,354692.500000,1.000000
max,404289.000000,537932.000000,537933.000000,1.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404290 non-null  int64 
 1   qid1          404290 non-null  int64 
 2   qid2          404290 non-null  int64 
 3   question1     404290 non-null  object
 4   question2     404290 non-null  object
 5   is_duplicate  404290 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


# **TF-IDF**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
documentA = 'the man went out for a walk'
documentB = 'the children sat around the fire'

In [ ]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([documentA, documentB])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df7 = pd.DataFrame(denselist, columns=feature_names)

In [ ]:
print(df7.head())

     around  children      fire      for      man      out       sat  \
0  0.000000  0.000000  0.000000  0.42616  0.42616  0.42616  0.000000   
1  0.407401  0.407401  0.407401  0.00000  0.00000  0.00000  0.407401   

        the     walk     went  
0  0.303216  0.42616  0.42616  
1  0.579739  0.00000  0.00000  


In [ ]:
print(vectors)

  (0, 8)	0.42615959880289433
  (0, 3)	0.42615959880289433
  (0, 5)	0.42615959880289433
  (0, 9)	0.42615959880289433
  (0, 4)	0.42615959880289433
  (0, 7)	0.3032160644503863
  (1, 2)	0.40740123733358447
  (1, 0)	0.40740123733358447
  (1, 6)	0.40740123733358447
  (1, 1)	0.40740123733358447
  (1, 7)	0.5797386715376657


In [ ]:
print(feature_names)

['around', 'children', 'fire', 'for', 'man', 'out', 'sat', 'the', 'walk', 'went']


In [ ]:
df['question1']

0         What is the step by step guide to invest in sh...
1         What is the story of Kohinoor (Koh-i-Noor) Dia...
2         How can I increase the speed of my internet co...
3         Why am I mentally very lonely? How can I solve...
4         Which one dissolve in water quikly sugar, salt...
                                ...                        
404285    How many keywords are there in the Racket prog...
404286            Do you believe there is life after death?
404287                                    What is one coin?
404288    What is the approx annual cost of living while...
404289                What is like to have sex with cousin?
Name: question1, Length: 404290, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
question = list(df['question1']) + list(df['question2'])

questions = [ques for ques in question if ques not in STOP_WORDS]

tfidf = TfidfVectorizer(lowercase=False, )
x=tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [ ]:
print(x) # first id , feautre_names no. , tfidf score

  (0, 77824)	0.28810089872803335
  (0, 83151)	0.27283405422911383
  (0, 97832)	0.2994105898351769
  (0, 77574)	0.20122105586038397
  (0, 78968)	0.2870081522008514
  (0, 103601)	0.09954570141748624
  (0, 74674)	0.3612041095478802
  (0, 60060)	0.19437825565580152
  (0, 100415)	0.6561773827369753
  (0, 103025)	0.07891022032825844
  (0, 79172)	0.0938889428826086
  (0, 51739)	0.07923245022121653
  (1, 16010)	0.44275849179086657
  (1, 34922)	0.497188073608617
  (1, 27864)	0.4741291764130098
  (1, 27867)	0.4632308754936714
  (1, 87457)	0.10928916808876182
  (1, 100638)	0.2895425035439381
  (1, 103025)	0.07944306353728867
  (1, 79172)	0.09452293028512647
  (1, 51739)	0.07976746929554651
  (2, 50118)	0.44691071060685306
  (2, 106157)	0.29128739079040816
  (2, 107880)	0.30756056894249173
  (2, 63504)	0.42337596422607426
  :	:
  (808573, 92091)	0.21004595937718346
  (808573, 59992)	0.16564744818997473
  (808573, 91680)	0.23467372994318145
  (808573, 74907)	0.19931796385379638
  (808573, 92159)	0.

In [ ]:
tfidf.idf_

array([10.04915306,  7.30458485, 13.90988277, ..., 13.50441767,
       13.50441767, 13.50441767])

In [ ]:
abf = tfidf.get_feature_names()
abf[10000:10020]

['BookRenter',
 'BookSellBuy',
 'Booker',
 'Booking',
 'BookinginBit',
 'Bookit',
 'Booklet',
 'Bookmarked',
 'Bookmarks',
 'Bookmyoffer',
 'Books',
 'Bookshelf',
 'Bookshelves',
 'Booktract',
 'Bool',
 'Boolean',
 'Boom',
 'Boomer',
 'Boomerang',
 'Boomers']

In [ ]:
word2tfidf['what']

4.767607133196319

In [ ]:
len(abf)==len(word2tfidf)

True

In [ ]:
d = documentA.split(' ')
print(d)

l = [word for word in d if word not in STOP_WORDS]
print(l)

['the', 'man', 'went', 'out', 'for', 'a', 'walk']
['man', 'went', 'walk']


In [ ]:
for i in l:
  print("{0}={1}".format(i,word2tfidf[i]))

man=6.6814943226847685
went=8.40455123832601
walk=8.692233310777791


## Xgboost

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(pd.concat((df['question1'],df['question2'])).unique())
trainq1_trans = tfidf_vect.transform(df['question1'].values)
trainq2_trans = tfidf_vect.transform(df['question2'].values)
labels = df['is_duplicate'].values
X = scipy.sparse.hstack((trainq1_trans,trainq2_trans))
y = labels
X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size = 0.33, random_state = 42)

xgb_model = xgb.XGBClassifier(n_estimators=150,random_state=42, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8).fit(X_train, y_train) 
xgb_prediction = xgb_model.predict(X_valid)

In [ ]:
from sklearn.metrics import f1_score, classification_report, accuracy_score
print(accuracy_score(y_valid,xgb_prediction))
print(classification_report(y_valid, xgb_prediction))

0.6904419260058764
              precision    recall  f1-score   support

           0       0.68      0.95      0.79     83975
           1       0.75      0.24      0.37     49441

    accuracy                           0.69    133416
   macro avg       0.72      0.60      0.58    133416
weighted avg       0.71      0.69      0.64    133416



In [ ]:
import pickle
filename = 'finalized_model1.sav'
pickle.dump(xgb_model, open(filename, 'wb'))

In [ ]:
X_valid.shape

(133416, 5000)

In [ ]:
print(X_valid[0].shape)

(1, 5000)


In [ ]:
filename = 'finalized_model1.sav'
# test=X_valid[105000]
result=loaded_model.predict(test)[0]
print(result)

TypeError: ignored

In [ ]:
filename = 'finalized_model1.sav'
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(X_valid)

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(pd.concat((df['question1'],df['question2'])).unique())

TfidfVectorizer(max_features=5000, token_pattern='\\w{1,}')

In [ ]:
question = list(df['question2']) + list(df['question1']) 
print(question[0])

What is the step by step guide to invest in share market?


In [ ]:
abc = tfidf_vect.transform(["Who would be better for India, Donald Trump or Hillary Clinton?"])
for i in tqdm(question):
  adc = tfidf_vect.transform([i])
  x=scipy.sparse.hstack((abc,adc))
  if loaded_model.predict(x)==1:
    print(i)

  0%|          | 8/808580 [00:00<5:39:21, 39.71it/s]

What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?
How can Internet speed be increased by hacking through DNS?
What keeps childern active and far from phone and video games?
How do I hack Motorola DCX3400 for free internet?
What are some of the things technicians can tell about the durability and reliability of Laptops and its components?
How can I see all my Youtube comments?


  0%|          | 27/808580 [00:00<3:00:47, 74.54it/s]

How can you make physics easy to learn?
What was your first sexual experience?
How will a Trump presidency affect the students presently in US or planning to study in US?
What does manipulation means?
Why do people ask Quora questions which can be answered easily by Google?
Which is the best digital marketing institute in Pune?
Why are rockets and boosters painted white?
What can I do to avoid being jealous of someone?
Which question should I ask on Quora?
What is the web application framework?
How do sports contribute to the society?
What is best way to ask for money online?
What's one thing you do despite knowing better?


  0%|          | 47/808580 [00:00<2:33:28, 87.81it/s]

...

  0%|          | 1931/808580 [00:12<2:10:37, 102.92it/s]

How does the ban on 500/1000 denominations affect various domains of the Indian economy?
Is buying WoW gold safe?
Which one is better IBM GBS or TCS BPS?
How can India justify keeping Kashmiris hostage for satisfying collective conscience of the jingoists?


  0%|          | 1941/808580 [00:12<1:28:50, 151.32it/s]

Which are the best free app downloading sites?
Is it possible to permanently delete a Quora question?
What are the pros and cons of client server network?


KeyboardInterrupt: ignored

In [ ]:
abc = tfidf_vect.transform(["How can i grow my muscles"])
adc = tfidf_vect.transform([""])
x=scipy.sparse.hstack((abc,adc))

In [ ]:
print(abc)
print(abc.shape)
print(adc.shape)

  (0, 2976)	0.2796941223694729
  (0, 2968)	0.6907832715552056
  (0, 2179)	0.1979886038778618
  (0, 2162)	0.18487849999298397
  (0, 1996)	0.5631868257286617
  (0, 748)	0.2324450944889363
(1, 5000)
(1, 5000)


In [ ]:
x=scipy.sparse.hstack((abc,adc))
print(x.shape)

(1, 10000)


In [ ]:
result=loaded_model.predict(x)
print(result)

[1]


## Random Forest

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(pd.concat((df['question1'],df['question2'])).unique())
trainq1_trans = tfidf_vect.transform(df['question1'].values)
trainq2_trans = tfidf_vect.transform(df['question2'].values)
labels = df['is_duplicate'].values
X = scipy.sparse.hstack((trainq1_trans,trainq2_trans))
y = labels
X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size = 0.33, random_state = 42)

clf = RandomForestClassifier(n_estimators=80).fit(X_train,y_train)
adb = clf.predict(X_valid)
print('word level tf-idf validation score:', f1_score(y_valid, clf.predict(X_valid), average='macro'))

In [ ]:
from sklearn.metrics import f1_score, classification_report, accuracy_score
print(accuracy_score(y_valid,adb))
print(classification_report(y_valid,adb))

0.8020402350542664
              precision    recall  f1-score   support

           0       0.79      0.93      0.86     83975
           1       0.83      0.58      0.69     49441

    accuracy                           0.80    133416
   macro avg       0.81      0.76      0.77    133416
weighted avg       0.81      0.80      0.79    133416



In [ ]:
import pickle
filename = '/content/drive/MyDrive/Capstone Project/Randomreg_tf_idf_model.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(pd.concat((df['question1'],df['question2'])).unique())
trainq1_trans = tfidf_vect.transform(df['question1'].values)
trainq2_trans = tfidf_vect.transform(df['question2'].values)
labels = df['is_duplicate'].values
X = scipy.sparse.hstack((trainq1_trans,trainq2_trans))
y = labels
X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(pd.concat((df['question1'],df['question2'])).unique())

TfidfVectorizer(max_features=5000, token_pattern='\\w{1,}')

In [ ]:
import pickle
filename = '/content/drive/MyDrive/Capstone Project/Random_tf_idf_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
abc = tfidf_vect.transform(["What is the story of Kohinoor (Koh-i-Noor) Diamond?"])
adc = tfidf_vect.transform(["who stole Kohinoor"])
x=scipy.sparse.hstack((abc,adc))

In [ ]:
# print(abc)
# print(adc)

In [ ]:
result=loaded_model.predict(x)
# print(type(result))
print(result)

[0]


In [ ]:
question = list(df['question2']) + list(df['question1'])
print(question[0])

What is the step by step guide to invest in share market?


In [ ]:
abc = tfidf_vect.transform(["Who will win the election? Donald Trump or Hillary Clinton?"])
for i in tqdm(question):
  adc = tfidf_vect.transform([i])
  x=scipy.sparse.hstack((abc,adc))
  if loaded_model.predict(x)==1:
    print(i)

In [ ]:
print(tfidf_vect.transform(["What are your views on ban of 500 and 1000 rupee notes in India?"]))

  (0, 4988)	0.20749418915035034
  (0, 4880)	0.09336759583274874
  (0, 4770)	0.3681885585546609
  (0, 3848)	0.3992635153498606
  (0, 3130)	0.1774574599331089
  (0, 3105)	0.12846408910157217
  (0, 3070)	0.34335636961859145
  (0, 2258)	0.21165653119667063
  (0, 2243)	0.1177790326312508
  (0, 483)	0.39588862658062457
  (0, 355)	0.1316904085058218
  (0, 287)	0.13877848377940755
  (0, 76)	0.34150415770363646
  (0, 7)	0.3445890334613565


## AdaBoost

In [ ]:
Ada_model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=80).fit(X_train, y_train) 
Ada_prediction = Ada_model.predict(X_valid)

In [ ]:
print(accuracy_score(y_valid,Ada_prediction))
print(classification_report(y_valid,Ada_prediction))

0.7239911255021887
              precision    recall  f1-score   support

           0       0.79      0.76      0.78     83975
           1       0.62      0.67      0.64     49441

    accuracy                           0.72    133416
   macro avg       0.71      0.71      0.71    133416
weighted avg       0.73      0.72      0.73    133416



# **Word2vector**

In [ ]:
import gensim
from gensim.models import Word2Vec
import multiprocessing


csv_path = "/content/drive/MyDrive/FYP/Hotel_Reviews.csv" #random dataset

df1 = pd.read_csv(csv_path)

In [ ]:
df1.head()

In [ ]:
df1.Description

In [ ]:
def remover(text):  #function for removing the Stop words
  text_t = text.split(" ")  
  l2 = [words.lower() for words in text_t if words not in STOP_WORDS]
  return l2;

In [ ]:
Description = df1.Description.apply(remover)
Description2 = df1.Description.apply(gensim.utils.simple_preprocess)
print(Description[0])
print(Description2[0])

In [ ]:
cores = multiprocessing.cpu_count()
print(cores)
model_sample = Word2Vec(window=5,min_count=4,workers = cores-1)
print(model_sample)

In [ ]:
model_sample.build_vocab(Description,progress_per=1000)

In [ ]:
print(model_sample)
model_sample.epochs

In [ ]:
model_sample.train(Description,total_examples=model_sample.corpus_count,epochs=10)

In [ ]:
model_sample.wv.most_similar("bad")

### Main DataSet

In [ ]:
question = list(df['question1'])

Using Stop_words list for preprocessing

In [ ]:
def clean_data(text):
    text = re.sub(r'[^ \nA-Za-z0-9À-ÖØ-öø-ÿ/]+', '', text)
    text = re.sub(r'[\\/×\^\]\[÷]', '', text)
    return text

In [ ]:
questions= []
for ques in question:
  ques = clean_data(ques)
  questions.append(remover(ques))


In [ ]:
print(questions[1:10])

In [ ]:
cores = multiprocessing.cpu_count()
print(cores)
model1 = Word2Vec(window=5,min_count=4,workers = cores-1)
print(model1)

In [ ]:
model1.build_vocab(questions,progress_per=10000)
print(model1)
model1.epochs

In [ ]:
model1.train(questions,total_examples=model1.corpus_count,epochs=5)

In [ ]:
model1.wv.most_similar("bad")

In [ ]:
model1.wv.similarity(w1="bad",w2="good")

In [ ]:
model1.wv.similarity(w1="bad",w2="bad")

In [ ]:
model1.wv.similarity(w1="bad",w2="horrible")

### Using in Built Preprocesser

In [ ]:
question2=[]
for ques in question:
  question2.append(gensim.utils.simple_preprocess(ques))

In [ ]:
print(question2[1:10])

In [ ]:
cores = multiprocessing.cpu_count()
print(cores)
model2 = Word2Vec(window=5,min_count=4,workers = cores-1)
print(model2)

In [ ]:
model2.build_vocab(question2,progress_per=10000)
print(model2)
model2.epochs

In [ ]:
model2.train(question2,total_examples=model2.corpus_count,epochs=5)

In [ ]:
model2.wv.most_similar("bad")

In [ ]:
model2.wv.similarity(w1="bad",w2="good")

In [ ]:
model2.wv.similarity(w1="bad",w2="bad")

### **Using Skip-Gram**

In [ ]:
cores = multiprocessing.cpu_count()
print(cores)
model_sg = Word2Vec(window=5,min_count=4,workers = cores-1,sg=3)
print(model_sg)

2
Word2Vec(vocab=0, size=100, alpha=0.025)


In [ ]:
model_sg.build_vocab(questions,progress_per=10000)
print(model_sg)
model_sg.epochs

Word2Vec(vocab=26440, size=100, alpha=0.025)


5

In [ ]:
model_sg.train(questions,total_examples=model_sg.corpus_count,epochs=4)

(9129460, 11032116)

In [ ]:
model_sg.wv.most_similar("bad")

In [ ]:
print(model_sg)

In [ ]:
model_sg.wv.most_similar("good")

### Checking Similiarty

In [ ]:
def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [ ]:
index2word_set = set(model.wv.index2word) 
s1_afv = avg_feature_vector(df['question1'][0], model=model_sg, num_features=100, index2word_set=index2word_set)
s2_afv = avg_feature_vector(df['question2'][60], model=model_sg, num_features=100, index2word_set=index2word_set)

In [ ]:
c

array([ 0.12988515,  0.19888775, -0.08141629,  0.36509874, -0.37354535,
       -0.12349963,  0.13633017,  0.15561588,  0.10642807, -0.15626147,
       -0.19083963,  0.34437352,  0.11409807, -0.03685695, -0.12273338,
        0.13084419,  0.20529003, -0.21268964,  0.3406952 , -0.00960251,
       -0.15573041, -0.5247952 ,  0.10925865, -0.38060603, -0.2747555 ,
        0.04763616,  0.18130836, -0.0170976 ,  0.01543326,  0.18248022,
        0.24445553,  0.18257365, -0.31695476,  0.11904132,  0.42668495,
       -0.26427427, -0.11535144, -0.18808846, -0.0806748 ,  0.02461837,
       -0.25562152,  0.14288247,  0.27904695,  0.2976209 ,  0.10477491,
       -0.00576999, -0.00400038, -0.51480067, -0.20681974,  0.2076648 ,
       -0.17604178, -0.08268839,  0.13639182,  0.15709622,  0.04054039,
       -0.42009935,  0.24349788, -0.2783216 ,  0.0127066 ,  0.25332743,
       -0.25645417, -0.08223865,  0.01918735, -0.12639834, -0.07310881,
       -0.0625005 ,  0.29395682,  0.07235251,  0.16495918, -0.04

In [ ]:
sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
print(sim)

1


In [ ]:
a = "Who would be better for India, Donald Trump or Hillary Clinton";

In [ ]:
data = {}
s1_afv = avg_feature_vector(a, model=model_sg, num_features=100, index2word_set=index2word_set)
for i,j in enumerate(df['question2']):
  s2_afv = avg_feature_vector(j, model=model_sg, num_features=100, index2word_set=index2word_set)
  sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
  if sim>0.9: print(f"{sim} = {j}")

1 = Is Kickass Torrents trustworthy?
1 = Is World War III imminent?
1 = What Is Web Design?
1 = DO YOU THINK IF MY TREE FALLS ON YOUR PROPERTY IT SHOULD BE MYINE OR YOUR RESPONSABILITY?
1 = Is Scientology bullshit?
1 = Polished vitrified tiles?
1 = Is President-elect Donald Trump President Putin's Trojan Horse?
1 = Is Narendra Modi obese?
1 = Is World War III coming?
1 = Are Near Death Experiences Proved?
1 = Is Hillary Clinton Pro-War?
1 = Was Robert Baratheon depressed?
1 = Is zerocensorship.com safe?
1 = Is Snapchat dead?
1 = Lip Balm: Is Carmex addictive?
1 = Is Ca tough?
1 = Who would be better for India, Donald Trump or Hillary Clinton?
1 = Does God exist?
1 = Is Quora relevant?
1 = Is Elizabeth Holmes married?
1 = Are Near Death Experiences (NDEs) real?
1 = Is Bernie Sanders gay?
1 = Which Is The Best SMS Sending Software?
1 = Am I dumb? 
1 = Do superhumans exist?
1 = Hillary Clinton nothingburger?
1 = Was Akbar Really Great?
1 = Is whistleblowing ethical?
1 = Is Batman insane?


KeyboardInterrupt: ignored

In [ ]:
data

# Bag of Word

In [ ]:
q1=df['question1']
q2=df['question2']
ques = np.concatenate((q1,q2),axis=0)
print(len(ques))

In [ ]:
vectorizer2= CountVectorizer()
vectorizer2.fit(ques)
print(len(vectorizer2.vocabulary_))

In [ ]:
bow_vector = vectorizer2.transform(ques)
print(bow_vector.shape)
print(bow_vector[0])

## Xgboost

In [ ]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(pd.concat((df['question1'],df['question2'])).unique())
trainq1_trans = count_vect.transform(df['question1'].values)
trainq2_trans = count_vect.transform(df['question2'].values)
labels = df['is_duplicate'].values
X = scipy.sparse.hstack((trainq1_trans,trainq2_trans))
y = labels
X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size = 0.33, random_state = 42)

xgb_model = xgb.XGBClassifier(n_estimators=80).fit(X_train, y_train)
xgb_prediction = xgb_model.predict(X_valid)

from sklearn.metrics import f1_score, classification_report, accuracy_score

print('training score:', f1_score(y_train, xgb_model.predict(X_train), average='macro'))
print('validation score:', f1_score(y_valid, xgb_model.predict(X_valid), average='macro'))
print(classification_report(y_valid, xgb_prediction))

training score: 0.5583329839621789
validation score: 0.5573457026584304
              precision    recall  f1-score   support

           0       0.67      0.96      0.79     83975
           1       0.77      0.20      0.32     49441

    accuracy                           0.68    133416
   macro avg       0.72      0.58      0.56    133416
weighted avg       0.71      0.68      0.62    133416



## Random Forest



In [ ]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(pd.concat((df['question1'],df['question2'])).unique())
trainq1_trans = count_vect.transform(df['question1'].values)
trainq2_trans = count_vect.transform(df['question2'].values)
labels = df['is_duplicate'].values
X = scipy.sparse.hstack((trainq1_trans,trainq2_trans))
y = labels
X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size = 0.33, random_state = 42)

clf = RandomForestClassifier(n_estimators=80).fit(X_train,y_train)
adb = clf.predict(X_valid)
print('word level tf-idf validation score:', f1_score(y_valid, clf.predict(X_valid), average='macro'))

word level tf-idf validation score: 0.7822300988561282


In [ ]:
from sklearn.metrics import f1_score, classification_report, accuracy_score
print(accuracy_score(y_valid,adb))
print(classification_report(y_valid,adb))

0.8103300953408886
              precision    recall  f1-score   support

           0       0.80      0.93      0.86     83975
           1       0.83      0.61      0.70     49441

    accuracy                           0.81    133416
   macro avg       0.82      0.77      0.78    133416
weighted avg       0.81      0.81      0.80    133416



## Ada Boost

In [ ]:
Ada_model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=80).fit(X_train, y_train)
Ada_prediction = Ada_model.predict(X_valid)

In [ ]:
print(accuracy_score(y_valid,Ada_prediction))
print(classification_report(y_valid,Ada_prediction))

0.7627271091923008
              precision    recall  f1-score   support

           0       0.76      0.92      0.83     83975
           1       0.79      0.49      0.61     49441

    accuracy                           0.76    133416
   macro avg       0.77      0.71      0.72    133416
weighted avg       0.77      0.76      0.75    133416

